### Cleaning Raw Tables - 1st Five - Aleem
(Reviewed and Organised by Caden)

In [ ]:
select * from CUSTOMERS_RAW;

-- EDITED BY CADEN:
-- Since table is alright, we recreate as new table 'CUSTOMERS_CLEANED'.
CREATE OR REPLACE TABLE CUSTOMERS_CLEANED AS
SELECT * FROM CUSTOMERS_RAW;

select * from CUSTOMERS_CLEANED;  --this is the final customer table

In [ ]:
select * from EMPLOYEE_RAW;

-- EDITED BY CADEN:
-- Since table is alright, we recreate as new table 'CUSTOMERS_CLEANED'.
CREATE OR REPLACE TABLE EMPLOYEE_CLEANED AS
SELECT * FROM EMPLOYEE_RAW;

select * from EMPLOYEE_CLEANED;   --this is the final employee table

In [ ]:
SELECT * FROM INVENTORY_TRANSACTIONS_RAW;

-- Step 1: Create a cleaned version of the table without unnecessary columns
CREATE OR REPLACE TABLE INVENTORY_TRANSACTIONS_CLEANED AS
SELECT
  TransactionID,
  ProductID,
  CAST(COALESCE(NULLIF(PurchaseOrderID, ''), -1) AS INT) AS PurchaseOrderID,
  COALESCE(
    TransactionDate,
    TO_DATE ('1900-01-01', 'yyyy-mm-DD')
  ) AS TransactionDate,
  CAST(
    COALESCE(NULLIF(UnitPurchasePrice, ''), 0) AS FLOAT
  ) AS UnitPurchasePrice,
  CAST(COALESCE(NULLIF(QuantityOrdered, ''), 0) AS INT) AS QuantityOrdered,
  CAST(COALESCE(NULLIF(QuantityReceived, ''), 0) AS INT) AS QuantityReceived,
  CAST(COALESCE(NULLIF(QuantityMissing, ''), 0) AS INT) AS QuantityMissing
FROM
  INVENTORY_TRANSACTIONS_RAW
WHERE
  NOT (MissingID IS NULL);

-- Step 2: Verify the cleaned data
SELECT *
FROM INVENTORY_TRANSACTIONS_CLEANED
LIMIT 10;

-- Step 3: Add indexes or constraints (optional)
-- For example, if TransactionID should be unique:
ALTER TABLE INVENTORY_TRANSACTIONS_CLEANED ADD PRIMARY KEY (TransactionID);  
--this is the final inventory transactions ID although there are still issues to be fixed.

SELECT * FROM INVENTORY_TRANSACTIONS_CLEANED;

In [ ]:
SELECT * FROM ORDERS_RAW;

-- Step 1: Create a cleaned version of the table
CREATE OR REPLACE TABLE ORDERS_CLEANED AS
SELECT
    ORDERID,
    CUSTOMERID,
    EMPLOYEEID,
    -- Replace NULL or empty strings with 0 and cast to INTEGER to remove decimal points
    CAST(COALESCE(NULLIF(SHIPPINGMETHODID, ''), 0) AS INTEGER) AS SHIPPINGMETHODID,
    ORDERDATE,
    SHIPDATE,
    FREIGHTCHARGE
FROM ORDERS_RAW;

-- Step 2: Verify the cleaned table
SELECT *
FROM ORDERS_CLEANED;

In [ ]:
select * from ORDER_DETAILS_RAW;

-- EDITED BY CADEN:
-- Since table is alright, we recreate as new table 'ORDER_DETAILS_CLEANED'.
CREATE OR REPLACE TABLE ORDER_DETAILS_CLEANED AS
SELECT * FROM ORDER_DETAILS_RAW;

select * from ORDER_DETAILS_CLEANED;  --this is the final order details table

### Cleaning Raw Tables - nxt Six - Hakim
(Reviewed and Organised by Caden)

In [ ]:
SELECT * FROM PRODUCTS_RAW;

-- Step 1: Validate the table for NULL or invalid values
SELECT *
FROM PRODUCTS_RAW
WHERE ProductName IS NULL
   OR Category IS NULL
   OR Weight IS NULL;

-- Step 2: If all data is valid, rename the table

-- EDITED BY CADEN:
-- Since table is alright, we recreate as new table 'PRODUCTS_CLEANED'.
CREATE OR REPLACE TABLE PRODUCTS_CLEANED AS
SELECT * FROM PRODUCTS_RAW;

SELECT * FROM PRODUCTS_CLEANED; -- This is the final products table.

In [ ]:
-- PURCHASE_ORDERS_RAW
SELECT * FROM PURCHASE_ORDERS_RAW;

-- Step 1: Validate the table for NULL or invalid values
SELECT *
FROM PURCHASE_ORDERS_RAW
WHERE PurchaseOrderID IS NULL
   OR SupplierID IS NULL
   OR ShippingMethodID IS NULL
   OR OrderDate IS NULL;

-- Step 2: If all data is valid, rename the table

-- EDITED BY CADEN:
-- Since table is alright, we recreate as new table 'PURCHASE_ORDERS_CLEANED'.
CREATE OR REPLACE TABLE PURCHASE_ORDERS_CLEANED AS
SELECT * FROM PURCHASE_ORDERS_RAW;

SELECT * FROM PURCHASE_ORDERS_CLEANED; -- This is the final purchase orders table.

In [ ]:
-- SHIPPING_METHODS_RAW
SELECT * FROM SHIPPING_METHODS_RAW;

-- Step 1: Validate the table for NULL or invalid values
SELECT *
FROM SHIPPING_METHODS_RAW
WHERE ShippingMethod IS NULL;

-- Step 2: If all data is valid, rename the table

-- EDITED BY CADEN:
-- Since table is alright, we recreate as new table 'SHIPPING_METHODS_CLEANED'.
CREATE OR REPLACE TABLE SHIPPING_METHODS_CLEANED AS
SELECT * FROM SHIPPING_METHODS_RAW;

SELECT * FROM SHIPPING_METHODS_CLEANED; -- This is the final shipping methods table.

In [ ]:
-- SUPPLIERS_RAW
SELECT * FROM SUPPLIERS_RAW;

-- Step 1: Validate the table for NULL or invalid values
SELECT *
FROM SUPPLIERS_RAW
WHERE SupplierName IS NULL;

-- Step 2: If all data is valid, rename the table

-- EDITED BY CADEN:
-- Since table is alright, we recreate as new table 'SUPPLIERS_CLEANED'.
CREATE OR REPLACE TABLE SUPPLIERS_CLEANED AS
SELECT * FROM SUPPLIERS_RAW;

SELECT * FROM SUPPLIERS_CLEANED; -- This is the final suppliers table.

In [ ]:
SELECT * FROM PAYMENTS_RAW;

-- EDITED BY CADEN: 
-- Step 1: Fix decimal places for PAYMENTAMOUNT
CREATE OR REPLACE TABLE PAYMENTS_CLEANED AS 
SELECT 
    PAYMENTID, 
    ORDERID, 
    PAYMENTMETHODID, 
    PAYMENTDATE, 
    TRY_TO_DECIMAL(REPLACE(PAYMENTAMOUNT, ','), 10, 2) AS PAYMENTAMOUNT 
FROM PAYMENTS_RAW 
WHERE PAYMENTID IS NOT NULL 
    AND ORDERID IS NOT NULL 
    AND PAYMENTMETHODID IS NOT NULL AND PAYMENTMETHODID != '' 
    AND PAYMENTDATE IS NOT NULL 
    AND PAYMENTAMOUNT IS NOT NULL;

-- Step 2: Verify the fixed table
SELECT *
FROM PAYMENTS_CLEANED;

In [ ]:
-- PAYMENT_METHODS_RAW
SELECT * FROM PAYMENT_METHODS_RAW;

-- Step 1: Validate the table for NULL or invalid values
SELECT *
FROM PAYMENT_METHODS_RAW
WHERE PaymentMethodID IS NULL
   OR PaymentMethod IS NULL;

-- Step 2: If there are no significant issues, rename the table

-- EDITED BY CADEN:
-- Since table is alright, we recreate as new table 'PAYMENT_METHODS_CLEANED'.
CREATE OR REPLACE TABLE PAYMENT_METHODS_CLEANED AS
SELECT * FROM PAYMENT_METHODS_RAW;

-- Step 3: Verify the renamed table
SELECT * FROM PAYMENT_METHODS_CLEANED;
-- This is the final payment methods table.

### Adding New Columns for Analysis - Caden

In [ ]:
-- ADD NEW COLUMN "DELIVERY_TIME" 
-- which is the number of days for the product to be shipped to customers 
ALTER TABLE ORDERS_CLEANED DROP COLUMN IF EXISTS DELIVERY_TIME;
ALTER TABLE ORDERS_CLEANED ADD COLUMN DELIVERY_TIME NUMBER DEFAULT 0;

UPDATE ORDERS_CLEANED
    SET DELIVERY_TIME = SHIPDATE - ORDERDATE;

-- check for results
SELECT TOP 10 * FROM ORDERS_CLEANED
ORDER BY DELIVERY_TIME ASC;

In [ ]:
-- Note: Not a new column but a row to handle orders with missing shipping method

-- ADD NEW ROW to SHIPPING_METHODS_CLEANED
-- for the rows that had a missing/NA shipping method
SELECT COUNT(ORDERID), SHIPPINGMETHODID 
FROM ORDERS_CLEANED
GROUP BY SHIPPINGMETHODID;

INSERT INTO SHIPPING_METHODS_CLEANED
VALUES (0, 'Unknown');

-- check for results
SELECT * FROM SHIPPING_METHODS_CLEANED;

In [ ]:
-- ADD NEW COLUMN "DAYS_TO_STOCK" 
-- which is the number of days for the items to reach our inventory after purchasing
ALTER TABLE INVENTORY_TRANSACTIONS_CLEANED DROP COLUMN IF EXISTS DAYS_TO_STOCK;
ALTER TABLE INVENTORY_TRANSACTIONS_CLEANED ADD COLUMN DAYS_TO_STOCK NUMBER DEFAULT 0;

UPDATE INVENTORY_TRANSACTIONS_CLEANED itc
    SET DAYS_TO_STOCK = poc.ORDERDATE - pc.INVENTORYDATE
FROM PURCHASE_ORDERS_CLEANED poc, PRODUCTS_CLEANED pc
WHERE itc.purchaseorderid = poc.purchaseorderid
AND itc.productid = pc.productid;

-- check for results
SELECT TOP 10 DAYS_TO_STOCK 
FROM INVENTORY_TRANSACTIONS_CLEANED;

In [ ]:
-- ADD NEW COLUMN "STOCK_LEVEL" 
-- which is the current remaining inventory of our products
ALTER TABLE PRODUCTS_CLEANED DROP COLUMN IF EXISTS STOCK_LEVEL;
ALTER TABLE PRODUCTS_CLEANED ADD COLUMN STOCK_LEVEL NUMBER DEFAULT 0;

UPDATE PRODUCTS_CLEANED pc
    SET STOCK_LEVEL = itc.total_received
FROM 
    (SELECT PRODUCTID, SUM(quantityreceived) total_received 
     FROM INVENTORY_TRANSACTIONS_CLEANED 
     GROUP BY PRODUCTID) itc
WHERE pc.productid = itc.productid;

-- in case none of the product is sold
UPDATE PRODUCTS_CLEANED pc
    SET STOCK_LEVEL = STOCK_LEVEL - odc.total_sold
FROM INVENTORY_TRANSACTIONS_CLEANED itc,
    (SELECT PRODUCTID, SUM(quantitysold) total_sold 
     FROM ORDER_DETAILS_CLEANED 
     GROUP BY PRODUCTID) odc
WHERE pc.productid = itc.productid
AND pc.productid = odc.productid;

-- check for results
SELECT * 
FROM PRODUCTS_CLEANED;

In [ ]:
-- ADD NEW COLUMN "VALUE_LOST" 
-- which is the amount of money lost from missing items
ALTER TABLE INVENTORY_TRANSACTIONS_CLEANED DROP COLUMN IF EXISTS VALUE_LOST;
ALTER TABLE INVENTORY_TRANSACTIONS_CLEANED ADD COLUMN VALUE_LOST NUMBER DEFAULT 0;

UPDATE INVENTORY_TRANSACTIONS_CLEANED
    SET VALUE_LOST = QUANTITYMISSING * UNITPURCHASEPRICE;

-- check for results
SELECT *
FROM INVENTORY_TRANSACTIONS_CLEANED
WHERE QUANTITYMISSING != 0;

In [ ]:
-- ADD NEW COLUMN "VALUE_RECEIVED" 
-- which is the value of the items received from purchasing
ALTER TABLE INVENTORY_TRANSACTIONS_CLEANED DROP COLUMN IF EXISTS VALUE_RECEIVED;
ALTER TABLE INVENTORY_TRANSACTIONS_CLEANED ADD COLUMN VALUE_RECEIVED NUMBER DEFAULT 0;

UPDATE INVENTORY_TRANSACTIONS_CLEANED
    SET VALUE_RECEIVED = QUANTITYRECEIVED * UNITPURCHASEPRICE;

-- check for results
SELECT TOP 5 *
FROM INVENTORY_TRANSACTIONS_CLEANED
WHERE UNITPURCHASEPRICE != 0;